# Hallucination detection progress using Flan-T5-Large
### Student Name:
Samin Adhikari

### Student ID:
223606554

### Traget Score:
Credit

### What's in this file.
- Progress upto benchmarking of normal vs parameterized model.
- Uses Hugging Face model downloaded using token and run locally.

In [69]:
# Install deps
!pip install transformers torch
!pip install evaluate
!pip install bert-score
!pip install absl-py
!pip install rouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=45680515c9d304beae616e4714261c35a468699c53960167922b5989d781fc1a
  Stored in directory: /Users/rvlife-samin/Library/Caches/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [77]:
# All the modules used for the project
import transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import evaluate
from huggingface_hub import login
from tqdm import tqdm

# Suppress logging
transformers.logging.set_verbosity_error()

In [54]:
# Hugging face token config
token = "<placeholder>"
login(token)

In [56]:
# Load model - default is google's flan-t5-large
def load_model_and_tokenizer(model_name="google/flan-t5-large"):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

# Default model used throuhout
def get_default_model_tokenizer():
    model_name = "google/flan-t5-large"
    model, tokenizer = load_model_and_tokenizer(model_name)
    return model, tokenizer

# Model with no specific hyperparameter
def generate_response(model, tokenizer, input_text, max_length=50):    
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(
      inputs["input_ids"],
      max_length=max_length
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Some parameter tuning along with prompt addition
def generate_parameterized_response(model, tokenizer, input_text, max_length=100, temperature=0.3):

    prompt = f"""
      Answer the following medical question using evidence-based clinical guidelines from trusted sources like PubMed, FDA, ACOG, WHO, etc. Provide relevant recommendations from these sources and explain the reasoning behind the answer. Include any conditions, risks, or precautions, and clearly state if further consultation is needed. If no evidence is available, explain the limitations of the available data.

      Question: {input_text}
    """
    
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
      inputs["input_ids"],
      max_length=max_length,
      temperature=temperature,
      do_sample=True, # Disables randomness for more reliable outputs
      num_beams=5, # Increases search space for high-quality answers
      repetition_penalty=1.2,  # Reduces hallucination by discouraging word overuse
      length_penalty=1.0,  # Maintains natural sentence flow
      early_stopping=True,  # Stops generation when an answer is complete
      num_return_sequences=1  # Returns only the best response
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [71]:
# Benchmarching
# Load all required Hugging Face metrics
bleu_metric = evaluate.load("bleu")
meteor_metric = evaluate.load("meteor")
rouge_metric = evaluate.load("rouge")
bertscore_metric = evaluate.load("bertscore")

def evaluate_metrics(predictions, references):
    # BLEU expects references as list of list of tokens
    bleu_result = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
    meteor_result = meteor_metric.compute(predictions=predictions, references=references)
    rouge_result = rouge_metric.compute(predictions=predictions, references=references, use_stemmer=True)
    bert_result = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

    return {
        "BLEU": bleu_result["bleu"],
        "METEOR": meteor_result["meteor"],
        "ROUGE-1": rouge_result["rouge1"],
        "ROUGE-L": rouge_result["rougeL"],
        "BERTScore-F1": sum(bert_result["f1"]) / len(bert_result["f1"]),
    }

def benchmark_model(generate_func, model, tokenizer, data):
    predictions, references = [], []

    for qa in tqdm(data):
        question = qa["question"]
        reference = qa["answer"]
        generated = generate_func(model, tokenizer, question)
        predictions.append(generated)
        references.append(reference)

    return evaluate_metrics(predictions, references)

def load_benchmark_data(path="data/question.json", limit=50):
    with open(path, "r") as f:
        data = json.load(f)
    return data[:limit]

[nltk_data] Downloading package wordnet to /Users/rvlife-
[nltk_data]     samin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/rvlife-
[nltk_data]     samin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/rvlife-
[nltk_data]     samin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [79]:
# Main execution step for the application

# Starts default unparameterized model
def start_default_qa_system():
    model, tokenizer = get_default_model_tokenizer()
    print("Welcome to the Medical QA system. Ask your medical question (or type 'exit' to quit).")
    
    while True:
        # Ask the user for a question
        input_text = input("Please enter your medical question: ")
        
        # Exit condition
        if input_text.lower() == "exit":
            print("Exiting the Medical QA system. Goodbye!")
            break
        
        # Generate and display the answer
        response = generate_response(model, tokenizer, input_text)
        print("\nAnswer:", response)
        print("\n---\n")

# Starts benchmarking of default vs parameterized model
def start_benchmark_system():
    print("Loading model...")
    model, tokenizer = get_default_model_tokenizer()

    print("Loading data...")
    data = load_benchmark_data()

    print("\n[1] Benchmarking default model...")
    default_results = benchmark_model(generate_response, model, tokenizer, data)
    print("\nDefault Results:")
    for k, v in default_results.items():
        print(f"{k}: {v:.4f}")

    print("\n[2] Benchmarking parameterized model...")
    param_results = benchmark_model(generate_parameterized_response, model, tokenizer, data)
    print("\nParameterized Results:")
    for k, v in param_results.items():
        print(f"{k}: {v:.4f}")

    print("\n--- COMPARISON ---")
    for metric in default_results:
        diff = param_results[metric] - default_results[metric]
        print(f"{metric}: Δ {diff:.4f} (Improvement: {'Yes' if diff > 0 else 'No'})")

# Main Call
print("Choose an option:")
print("1: Load default model")
print("2: Benchmark default and parameterized model")
choice = input("Enter the number corresponding to your choice: ")

if choice == '1':
    start_default_qa_system()
elif choice == '2':
    start_benchmark_system()
else:
    print("Invalid choice! Please enter a valid number.")

Choose an option:
1: Load default model
2: Benchmark default and parameterized model


Enter the number corresponding to your choice:  2


Loading model...
Loading data...

[1] Benchmarking default model...


100%|███████████████████████████████████████████| 50/50 [00:48<00:00,  1.02it/s]



Default Results:
BLEU: 0.0038
METEOR: 0.0577
ROUGE-1: 0.1205
ROUGE-L: 0.1086
BERTScore-F1: 0.8448

[2] Benchmarking parameterized model...


100%|███████████████████████████████████████████| 50/50 [08:45<00:00, 10.50s/it]



Parameterized Results:
BLEU: 0.0512
METEOR: 0.2239
ROUGE-1: 0.2532
ROUGE-L: 0.1998
BERTScore-F1: 0.8727

--- COMPARISON ---
BLEU: Δ 0.0475 (Improvement: Yes)
METEOR: Δ 0.1662 (Improvement: Yes)
ROUGE-1: Δ 0.1326 (Improvement: Yes)
ROUGE-L: Δ 0.0912 (Improvement: Yes)
BERTScore-F1: Δ 0.0279 (Improvement: Yes)
